# Integrated Gradients Benchmark

In [1]:
import sys
sys.path.append("../")

import fastISM
from fastISM.models.basset import basset_model

from fastISM.models.factorized_basset import factorized_basset_model
from fastISM.models.bpnet import bpnet_model
import tensorflow as tf
import numpy as np
from importlib import reload
import time

In [2]:
reload(fastISM.flatten_model)
reload(fastISM.models)
reload(fastISM.ism_base)
reload(fastISM.change_range)
reload(fastISM.fast_ism_utils)
reload(fastISM)

<module 'fastISM' from '../fastISM/__init__.py'>

In [3]:
tf.__version__

'2.3.0'

In [4]:
!nvidia-smi

Tue Sep  8 09:53:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:82:00.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-0d9a859c-ce19-78f3-2f87-aade11d14bae)


In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Apr_24_19:10:27_PDT_2019
Cuda compilation tools, release 10.1, V10.1.168


In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
device = 'GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/device:CPU:0'
device

'GPU:0'

In [9]:
import alibi
from alibi.explainers import IntegratedGradients

In [10]:
alibi.__version__

'0.5.4'

## Benchmark

In [11]:
def time_ig(model, batch_sizes, seqlen, num_examples=500, n_steps=50, targets = [None]):
    x = np.random.random((num_examples,seqlen,4))
    times = []
    per_100 = []
    for b in batch_sizes:
        ig  = IntegratedGradients(model,
                              layer=None,
                              method="gausslegendre",
                              n_steps=n_steps,
                              internal_batch_size=b)
        # dry run
        ig.explain(x[:10], baselines=None,
                   target=targets[0])
        
        t = time.time()
        for tgt in targets:
            ig.explain(x, baselines=None,
                       target=tgt)
        times.append(time.time()-t)
        per_100.append((times[-1]/num_examples)*100)
        print("BATCH: {}\tTIME: {:.2f}\tPER 100: {:.2f}".format(b, times[-1], (times[-1]/num_examples)*100))
    
    print("BEST PER 100: {:.2f}".format(min(per_100)))

### Basset (1000)

In [12]:
model = basset_model(seqlen=1000, num_outputs=1)

In [13]:
%%capture --no-stdout 
# hide warning about scalar output

time_ig(model, [100, 200, 500], 1000, num_examples=10, targets=[None]) # targets None since only one scalar output

BATCH: 100	TIME: 0.26	PER 100: 2.62
BATCH: 200	TIME: 0.24	PER 100: 2.37
BATCH: 500	TIME: 0.25	PER 100: 2.50
BEST PER 100: 2.37


In [14]:
%%capture --no-stdout

time_ig(model, [100, 200, 500, 1000], 1000, num_examples=100, targets=[None]) 

BATCH: 100	TIME: 2.77	PER 100: 2.77
BATCH: 200	TIME: 2.39	PER 100: 2.39
BATCH: 500	TIME: 2.34	PER 100: 2.34
BATCH: 1000	TIME: 3.28	PER 100: 3.28
BEST PER 100: 2.34


### Basset (2000)

In [12]:
model = basset_model(seqlen=2000, num_outputs=1)

In [13]:
%%capture --no-stdout

time_ig(model, [100, 200, 500], 2000, num_examples=10, targets=[None]) 

BATCH: 100	TIME: 0.48	PER 100: 4.80
BATCH: 200	TIME: 0.48	PER 100: 4.76
BATCH: 500	TIME: 0.48	PER 100: 4.82
BEST PER 100: 4.76


In [14]:
%%capture --no-stdout

time_ig(model, [100, 200, 500], 2000, num_examples=100, targets=[None]) 

BATCH: 100	TIME: 4.77	PER 100: 4.77
BATCH: 200	TIME: 4.63	PER 100: 4.63
BATCH: 500	TIME: 4.61	PER 100: 4.61
BEST PER 100: 4.61


### Factorized Basset (1000)

In [12]:
model = factorized_basset_model(seqlen=1000, num_outputs=1)

In [13]:
%%capture --no-stdout
time_ig(model, [100, 200, 500], 1000, num_examples=10, targets=[None])

BATCH: 100	TIME: 0.54	PER 100: 5.40
BATCH: 200	TIME: 0.51	PER 100: 5.14
BATCH: 500	TIME: 0.50	PER 100: 5.00
BEST PER 100: 5.00


In [14]:
%%capture --no-stdout
time_ig(model, [100, 200, 500], 1000, num_examples=100, targets=[None])

BATCH: 100	TIME: 5.18	PER 100: 5.18
BATCH: 200	TIME: 4.92	PER 100: 4.92
BATCH: 500	TIME: 4.82	PER 100: 4.82
BEST PER 100: 4.82


### Factorized Basset (2000)

In [12]:
model = factorized_basset_model(seqlen=2000, num_outputs=1)

In [13]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=10, targets=[None])

BATCH: 100	TIME: 0.99	PER 100: 9.89
BATCH: 200	TIME: 0.97	PER 100: 9.73
BATCH: 300	TIME: 0.95	PER 100: 9.47
BEST PER 100: 9.47


In [14]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=100, targets=[None])

BATCH: 100	TIME: 9.90	PER 100: 9.90
BATCH: 200	TIME: 9.56	PER 100: 9.56
BATCH: 300	TIME: 9.53	PER 100: 9.53
BEST PER 100: 9.53


In [15]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=200, targets=[None])

BATCH: 100	TIME: 19.84	PER 100: 9.92
BATCH: 200	TIME: 19.14	PER 100: 9.57
BATCH: 300	TIME: 19.07	PER 100: 9.54
BEST PER 100: 9.54


### BPNet (1000)

In [12]:
model = bpnet_model(seqlen=1000, num_dilated_convs=9)

# flatten and concat outputs
inp = tf.keras.Input(shape=model.input_shape[1:])
prof, cts = model(inp)
prof = tf.keras.layers.Flatten()(prof)
cts = tf.keras.layers.Flatten()(cts)
out = tf.keras.layers.Concatenate()([prof, cts])
model_ig = tf.keras.Model(inputs=inp, outputs=out)

# flattened outputs
model = model_ig

In [13]:
model.output

<tf.Tensor 'concatenate/concat:0' shape=(None, 1001) dtype=float32>

In [14]:
time_ig(model, [500], 1000, num_examples=10, targets=range(1001)) # all 1000 profile outs + 1 count out

BATCH: 500	TIME: 439.95	PER 100: 4399.53
BEST PER 100: 4399.53


### BPNet (2000)

In [12]:
model = bpnet_model(seqlen=2000, num_dilated_convs=9)

# flatten and concat outputs
inp = tf.keras.Input(shape=model.input_shape[1:])
prof, cts = model(inp)
prof = tf.keras.layers.Flatten()(prof)
cts = tf.keras.layers.Flatten()(cts)
out = tf.keras.layers.Concatenate()([prof, cts])
model_ig = tf.keras.Model(inputs=inp, outputs=out)

# flattened outputs
model = model_ig

In [13]:
model.output

<tf.Tensor 'concatenate/concat:0' shape=(None, 2001) dtype=float32>

In [14]:
time_ig(model, [500], 2000, num_examples=5, targets=range(2001)) # all 2000 profile outs + 1 count out

BATCH: 500	TIME: 622.02	PER 100: 12440.47
BEST PER 100: 12440.47
